# PaLM API：使用 Python 微调快速入门
在本笔记本中，您将了解如何使用 PaLM API 的 Python 客户端库开始使用调整服务。在这里，您将学习如何调整 PaLM API 文本生成服务背后的文本模型。
```{tip}
注意：目前，调整仅适用于 text-bison-001 模型。
```

## 安装
### 认证
PaLM API 允许您根据自己的数据调整模型。由于这是您的数据和调整后的模型，因此需要比 API 密钥提供的更严格的访问控制。

在运行本教程之前，您需要[为您的项目设置 OAuth](https://ai.google.dev/palm_docs/oauth_quickstart)。

如果您想在 Colab 中运行此笔记本，请首先使用“文件 > 上传”选项上传`client_secret*.json`文件。
```shell
cp client_secret*.json client_secret.json
ls client_secret.json
```
此 gcloud 命令将 client_secret.json 文件转换为可用于对服务进行身份验证的凭据。

```{importance}
重要提示：如果您在 Colab 中运行此程序，请不要仅单击它打印的链接。那会失败的。按照说明操作，将其打印的 gcloud 命令复制到本地计算机并在那里运行，然后将本地计算机的输出粘贴回此处。
```

```python
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

```

### 安装客户端库

In [1]:
pip install -q google-generativeai

Note: you may need to restart the kernel to use updated packages.


### 导入客户端库

In [9]:
import google.generativeai as genai

In [10]:
GOOGLE_API_KEY = "YOUR-API-KEY"
genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
import pprint
for model in genai.list_models():
    pprint.pprint(model)


Model(name='models/chat-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 Chat (Legacy)',
      description='A legacy text-only model optimized for chat conversations',
      input_token_limit=4096,
      output_token_limit=1024,
      supported_generation_methods=['generateMessage', 'countMessageTokens'],
      temperature=0.25,
      top_p=0.95,
      top_k=40)
Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)
Model(name='models/embedding-gecko-001',
      base_model_id='',
      version='001',
      display_name='Embedding Gecko',
      description='Obtain a distributed representatio

您可以使用 genai.list_tuned_model 方法检查现有的调整模型。

In [ ]:
for i, m in zip(range(5), genai.list_tuned_models()):
  print(m.name)

## 创建调整模型
要创建调整模型，您需要将数据集传递给`genai.create_tuned_model`方法中的模型。您可以通过直接定义调用中的输入和输出值或从文件导入数据帧以传递给方法来执行此操作。

对于此示例，您将调整模型以生成序列中的下一个数字。例如，如果输入为 1，则模型应输出 2。如果输入为 100，则输出应为 101。

In [12]:
base_model = [
    m for m in genai.list_models()
    if "createTunedTextModel" in m.supported_generation_methods][0]
base_model.name


'models/text-bison-001'

In [ ]:
import random

name = f'generate-num-{random.randint(0,10000)}'
operation = genai.create_tuned_model(
    # You can use a tuned model here too. Set `source_model="tunedModels/..."`
    source_model=base_model.name,
    training_data=[
        {
             'text_input': '1',
             'output': '2',
        },{
             'text_input': '3',
             'output': '4',
        },{
             'text_input': '-3',
             'output': '-2',
        },{
             'text_input': 'twenty two',
             'output': 'twenty three',
        },{
             'text_input': 'two hundred',
             'output': 'two hundred one',
        },{
             'text_input': 'ninety nine',
             'output': 'one hundred',
        },{
             'text_input': '8',
             'output': '9',
        },{
             'text_input': '-98',
             'output': '-97',
        },{
             'text_input': '1,000',
             'output': '1,001',
        },{
             'text_input': '10,100,000',
             'output': '10,100,001',
        },{
             'text_input': 'thirteen',
             'output': 'fourteen',
        },{
             'text_input': 'eighty',
             'output': 'eighty one',
        },{
             'text_input': 'one',
             'output': 'two',
        },{
             'text_input': 'three',
             'output': 'four',
        },{
             'text_input': 'seven',
             'output': 'eight',
        }
    ],
    id = name,
    epoch_count = 100,
    batch_size=4,
    learning_rate=0.001,
)


您的调整后的模型会立即添加到调整后的模型列表中，但在模型调整时其状态会设置为“正在创建”。

In [ ]:
model = genai.get_tuned_model(f'tunedModels/{name}')

model

In [ ]:
model.state

### 检查调整进度
使用`元数据`检查状态：

In [ ]:
operation.metadata

使用`operation.result()`或`operation.wait_bar()`等待训练完成

In [ ]:
import time

for status in operation.wait_bar():
  time.sleep(30)

您可以随时使用 cancel() 方法取消调整作业。取消注释下面的行并运行代码单元以在作业完成之前取消作业。

In [ ]:
# operation.cancel()

调优完成后，您可以从调优结果中查看损耗曲线。[损失曲线](https://generativeai.devsite.corp.google.com/guide/model_tuning_guidance#recommended_configurations)显示模型的预测与理想输出的偏差程度。

In [ ]:
import pandas as pd
import seaborn as sns

model = operation.result()

snapshots = pd.DataFrame(model.tuning_task.snapshots)

sns.lineplot(data=snapshots, x = 'epoch', y='mean_loss')

## 评估您的模型
您可以使用`genai.generate_text`方法并指定模型名称来测试模型性能。

In [ ]:
completion = genai.generate_text(model=f'tunedModels/{name}',
                                prompt='5')
completion.result

In [ ]:
completion = genai.generate_text(model=f'tunedModels/{name}',
                                prompt='-9')
completion.result

In [ ]:
completion = genai.generate_text(model=f'tunedModels/{name}',
                                prompt='four')
completion.result

正如您所看到的，最后一个提示没有产生理想的结果，即`5`。为了产生更好的结果，您可以尝试一些不同的方法，例如将温度调整为接近零以获得更一致的结果，向数据集中添加更多质量示例以供模型学习，或者向示例添加提示或序言。

有关提高性能的更多指导，请参阅[调优指南](https://generativeai.devsite.corp.google.com/guide/model_tuning_guidance)。

## 更新描述
您可以随时使用`genai.update_tuned_model`方法更新调整模型的描述。

In [ ]:
genai.update_tuned_model(f'tunedModels/{name}', {"description":"This is my model."})

In [ ]:
model = genai.get_tuned_model(f'tunedModels/{name}')

model

model.description

## 删除模型
您可以通过删除不再需要的模型来清理调整后的模型列表。使用`genai.delete_tuned_model`方法删除模型。如果您取消了任何调整作业，您可能需要删除这些作业，因为它们的性能可能无法预测。

In [ ]:
genai.delete_tuned_model(f'tunedModels/{name}')

try:
  m = genai.get_tuned_model(f'tunedModels/{name}')
  print(m)
except Exception as e:
  print(f"{type(e)}: {e}")